In [1]:
# Convert .tsv file to dataturks json format.

import json
import logging
import os

def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(os.getcwd() +  r'%s'%input_path, 'r', encoding = 'utf-8') # input file
        fp=open(os.getcwd() +  r'%s'%output_path, 'w+', encoding = 'utf-8') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        next(f)
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

In [3]:
os.chdir('../')

In [7]:
# Run once for Training data, and again for Evaluation data
# Change input/output directories/filenames above accordingly
te=['evaluation','training']
for i in te:
    in_file = '/NER/sentence_samples/tsv/%s_1.tsv'%i
    out_file = '/NER/sentence_samples/tsv/%s_1.json'%i
    tsv_to_json_format(in_file, out_file,'abc')